In [28]:
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
import glob
import os
from PIL import Image

In [29]:
def load_model(model_path):
    """Load a pre-trained TensorFlow model."""
    return tf.keras.models.load_model(model_path)

def load_npz_data(file_path):
    """Load ground truth raster from NPZ file."""
    with np.load(file_path, allow_pickle=True) as data:
        raster_data = data['arr_1']
        # print(data['arr_0'])
        # print("-"*45)
        # print(data["arr_1"])
    
    # print(raster_data)
    return raster_data

def inspect_npz_file(file_path):
    with np.load(file_path) as data:
        print(data.files)

# inspect_npz_file(r"C:\Users\lyq09mow\Data\Berlin\berlin-de\ground_truth_class_raster_25km.npz")
# load_npz_data(r"C:\Users\lyq09mow\Data\Berlin\berlin-de\ground_truth_class_raster_25km.npz")


In [30]:
def load_image(file_path):
    """Load an image file and convert it to a numpy array."""
    with Image.open(file_path) as img:
        return np.array(img)

In [31]:
def preprocess_image(image_array):
    """Preprocess image data for model prediction. Adjust based on model requirements."""
    return image_array.astype(np.float32) / 255.0


def predict_raster(model, raster_data):
    """Predict classes for each pixel in raster using the TensorFlow model."""
    raster_data = np.expand_dims(raster_data, axis=0)  # Add a batch dimension if needed
    predictions = model.predict(raster_data)
    return np.argmax(predictions, axis=-1).squeeze()  # Remove the batch dimension

In [32]:
def plot_rasters(raster1, raster2, title1='Ground Truth', title2='Prediction'):
    fig, axes = plt.subplots(1, 2, figsize=(12, 6))
    for ax, raster, title in zip(axes, [raster1, raster2], [title1, title2]):
        ax.imshow(raster, cmap='viridis')
        ax.set_title(title)
    plt.show()

In [33]:
def get_grid_sources(base_path, phase='testing'):
    # Collect all image file paths
    file_pattern = os.path.join(base_path, phase, '*', '*.jpg')
    files = glob.glob(file_pattern, recursive=True)
    
    # Create a DataFrame from the file paths
    files_df = pd.DataFrame(files, columns=['filename'])
    
    # Extract class, city, and phase information
    files_df['class'] = files_df['filename'].apply(lambda x: x.split(os.sep)[-2])
    files_df['city'] = files_df['filename'].apply(lambda x: x.split(os.sep)[-1].split('_')[0])
    
    return files_df

# Example usage
# base_path = r'C:\Users\lyq09mow\ModelImages\Bar_Ber_Mad_Rom'
# classes = ['Airports', 'Fast transit roads and associated land', 
#            'Forests', 'Green Urban Areas', 
#            'Industrial, commercial, public, military and private units', 
#            'Land without current use', 
#            'Railways and associated land', 
#            'Sports and leisure facilities',
#            'Urban Fabric', 'Water']  

# grid_sources_df = get_grid_sources(base_path, phase='testing')
# print(grid_sources_df.head())


In [38]:
import numpy as np

# Load the ground truth data
def load_npz_data(file_path, key='arr_0'):
    """Load ground truth raster from NPZ file using the specified key."""
    with np.load(file_path, allow_pickle=True) as data:
        raster_data = data[key]
    return raster_data

# Path to your ground truth NPZ file
ground_truth_path = r'C:\Users\lyq09mow\Data\Berlin\berlin-de\ground_truth_class_raster_25km.npz'

# Load the ground truth data
ground_truth_data = load_npz_data(ground_truth_path)

# Check the dimensions of the ground truth data
print("Ground truth data shape:", ground_truth_data.shape)


Ground truth data shape: (100, 100, 19)


In [ ]:
# File paths and model path
model_path = r'C:\Users\lyq09mow\Model\Bar_Ber_Mad_Rom\85cent9.keras'
ground_truth_path = r'C:\Users\lyq09mow\Data\Berlin\berlin-de\ground_truth_class_raster_25km.npz'
base_path = r'C:\Users\lyq09mow\ModelImages\Bar_Ber_Mad_Rom'
    
# Load the model
model = load_model(model_path)

# Load and process ground truth data
ground_truth_data = load_npz_data(ground_truth_path)

# Get grid sources for validation/testing
grid_sources_df = get_grid_sources(base_path, phase='testing')
    
# Load and preprocess input data for prediction
input_raster_path = grid_sources_df.iloc[0]['filename']  # Example with the first file; adapt as needed
input_raster_data = preprocess_image(load_image(input_raster_path))
    
# Generate predictions
predicted_classes = predict_raster(model, input_raster_data)

# Visualization
plot_rasters(ground_truth_data.argmax(axis=3), predicted_classes, title1='Ground Truth', title2='Prediction')